In [29]:
from pyspark.sql.functions import udf, explode
from pyspark.sql.functions import collect_list
from pyspark.sql.functions import col, concat_ws

StatementMeta(sparkPool, 0, 30, Finished, Available, Finished)

In [30]:
#notebook parameters. Leave them empty
in_file_genres = "bronze/tmdb/tmdb_genres_20250210.json"
in_file_trending = "bronze/tmdb/tmdb_trending_item_20250210.json"

StatementMeta(sparkPool, 0, 31, Finished, Available, Finished)

In [31]:
#TODO: uncomment the row below and assign your data lake storage name to the variable
storage_account = "stdimentoringdatalakeed"

container = "data"

date_postfix = in_file_genres[in_file_genres.find('.json') - 8 : in_file_genres.find('.json')]

file_genres = f"abfss://{container}@{storage_account}.dfs.core.windows.net/{in_file_genres}"

#TODO: uncomment the row below and assign movies trending file path to the ariable
file_trending = f"abfss://{container}@{storage_account}.dfs.core.windows.net/{in_file_trending}"

StatementMeta(sparkPool, 0, 32, Finished, Available, Finished)

## Load list of genres to a dataframe

In [32]:
%%pyspark
#TODO: uncomment the row below and complete missing parts with your code to create spark dataframe from file_genres source using spark.read.load method
df_genres = spark.read.json(file_genres)

#explode genres array
df_genres = df_genres.select(explode('genres').alias('genres')).select('genres.id','genres.name')

StatementMeta(sparkPool, 0, 33, Finished, Available, Finished)

## Load list of trending movies to a dataframe

In [33]:
%%pyspark
#TODO: uncomment the row below and complete missing parts with your code to create spark dataframe from file_trending source using spark.read.load method
df_movies = spark.read.json(file_trending)

#explode "results" property array
df_movies = df_movies.select(explode('results').alias('movies')).select('movies.id', 'movies.title', 'movies.popularity', 'movies.vote_average', 'movies.genre_ids')

#explode "genres_ids" property array
df_movies = df_movies.select(df_movies.id.alias('movie_id'), 'title', 'popularity', 'vote_average', explode('genre_ids').alias('genre_id'))

StatementMeta(sparkPool, 0, 34, Finished, Available, Finished)

In [34]:
#joining movies with genres dictionary to get genre name
inner_join = df_movies.join(df_genres,
     (df_movies.genre_id == df_genres.id), how = 'inner').select(df_movies.movie_id, df_movies.title, df_movies.popularity, df_movies.vote_average, df_genres.name.alias('genre_name'))

#collect movie genres to a list of strings
grouped_genres = inner_join.groupBy('movie_id', 'title', 'popularity', 'vote_average').agg(collect_list('genre_name').alias('genres'))

#get concatenated string value from list of genres
result = grouped_genres.withColumn("genres",
   concat_ws(",",col("genres")))     

StatementMeta(sparkPool, 0, 35, Finished, Available, Finished)

In [35]:
#write resulted dataframe to a parquet file in silver layer
write_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/tmdb/tmdb_trending_{date_postfix}"

#TODO: uncomment the row below and complete missing parts with your code to save "result" dataframe to data lake in parquet format using overwrite moe
result.write.parquet(write_path)

StatementMeta(sparkPool, 0, 36, Finished, Available, Finished)